In [1]:
import tensorflow as tf
import keras
from keras import backend as K

cfg = K.tf.ConfigProto(device_count = {'CPU' : 1, 'GPU' : 1})
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.utils import normalize
from keras.applications import ResNet50
from keras.callbacks import TensorBoard, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

import pickle
import time
from sklearn.model_selection import train_test_split
from LR_Finder import LR_Finder

Using TensorFlow backend.


In [2]:
PATH = "data/keras/cat_dog/"

In [3]:
train_data_dir = f'{PATH}TRAIN'
test_data_dir = f'{PATH}TEST'
batch_size = 64
sz = 224 #Use 224 since Resnet50 was traing on 224 and 299

In [4]:
train_datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1,
                                   height_shift_range=0.1, shear_range=0.15,
                                   zoom_range=0.1, channel_shift_range=10., horizontal_flip=True)
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(sz,sz),
                                                   batch_size=batch_size,
                                                   class_mode='binary')

test_generator = train_datagen.flow_from_directory(test_data_dir,
                                                   shuffle=False,
                                                   target_size=(sz,sz),
                                                   batch_size=batch_size,
                                                   class_mode='binary')

Found 19995 images belonging to 2 classes.
Found 5002 images belonging to 2 classes.


In [5]:
base_model = ResNet50(weights='imagenet', include_top=False)

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
base_model.summary()

In [6]:
# add layers to the base resnet50 model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [7]:
model = Model(inputs=base_model.input, outputs=predictions)
tensorboard = TensorBoard(log_dir='logs_resnet50_1/{}'.format("resnet_last_3_{}".format(int(time.time()))))
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=1,
                    validation_data=test_generator, validation_steps=test_generator.n // batch_size,
                    callbacks=[tensorboard])

Epoch 1/3
 27/312 [=>............................] - ETA: 3:45 - loss: 1.0310 - acc: 0.8397

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262146 bytes but only got 0. Skipping tag 2
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 262151 bytes but only got 0. Skipping tag 56
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 0. 
  warnings.warn(str(msg))


 58/312 [====>.........................] - ETA: 3:17 - loss: 0.6368 - acc: 0.8715

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18350080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


 82/312 [======>.......................] - ETA: 2:58 - loss: 0.4992 - acc: 0.8908

...

183/312 [================>.............] - ETA: 1:39 - loss: 0.3113 - acc: 0.9214

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 209715200 bytes but only got 0. Skipping tag 48
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 52428800 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6468665344 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1050744 bytes but only got 7027. Skipping tag 48
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-pac

237/312 [=====================>........] - ETA: 57s - loss: 0.2770 - acc: 0.9283

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.

312/312 [==============================] - 291s 932ms/step - loss: 0.2395 - acc: 0.9347 - val_loss: 1.8769 - val_acc: 0.5010
Epoch 2/3
312/312 [==============================] - 271s 869ms/step - loss: 0.1032 - acc: 0.9616 - val_loss: 1.3708 - val_acc: 0.5065
Epoch 3/3
312/312 [==============================] - 270s 866ms/step - loss: 0.0886 - acc: 0.9659 - val_loss: 2.8005 - val_acc: 0.5065
Wall time: 13min 53s


In [9]:
split_at = 140
model = Model(inputs=base_model.input, outputs=predictions)
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
tensorboard = TensorBoard(log_dir='logs_resnet50_1/{}'.format("resnet_last_140Split_{}".format(int(time.time()))))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])  

In [10]:
%%time
model.fit_generator(train_generator, train_generator.n // batch_size, epochs=3, workers=1,
                    validation_data=test_generator, validation_steps=test_generator.n // batch_size,
                    callbacks=[tensorboard])

Epoch 1/3
 25/312 [=>............................] - ETA: 3:40 - loss: 0.4360 - acc: 0.9094

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.

 39/312 [==>...........................] - ETA: 3:25 - loss: 0.3262 - acc: 0.9251

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80000 bytes but only got 0. Skipping tag 64640
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 307363840 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 307888128 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\P

172/312 [===============>..............] - ETA: 1:42 - loss: 0.1450 - acc: 0.9575

...

270/312 [========================>.....] - ETA: 30s - loss: 0.1162 - acc: 0.9649

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18350080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:771: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


305/312 [============================>.] - ETA: 5s - loss: 0.1113 - acc: 0.9659

c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 404094976 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 404619264 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 425459712 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-packages\PIL\TiffImagePlugin.py:754: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1385474 bytes but only got 6833. Skipping tag 513
  " Skipping tag %s" % (size, len(data), tag))
c:\users\tomesd\anaconda3\envs\keras\lib\site-pac

312/312 [==============================] - 279s 893ms/step - loss: 0.1108 - acc: 0.9660 - val_loss: 4.8898 - val_acc: 0.6745
Epoch 2/3
312/312 [==============================] - 271s 868ms/step - loss: 0.0564 - acc: 0.9819 - val_loss: 0.8815 - val_acc: 0.9237
Epoch 3/3
312/312 [==============================] - 271s 868ms/step - loss: 0.0403 - acc: 0.9860 - val_loss: 0.9277 - val_acc: 0.9111
Wall time: 13min 44s


In [ ]:
model.summary()

In [ ]:
lr_finder = LR_Finder(start_lr=1e-5, end_lr=10, step_size=np.ceil(X_train.shape[0]/64))
model.fit(X_train, y_train, callbacks=[lr_finder] )

In [ ]:
lr_finder.plot_lr()

In [ ]:
lr_finder.plot()

In [ ]:
lr_finder.plot_smoothed_loss()